<a href="https://colab.research.google.com/github/Tejaswini-44/Covid-19-Pneumonia-and-Tuberculosis-disease-detection-by-the-processing-of-XRAY/blob/main/XRAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
imageSize = [224, 224]

trainPath = r"/content/drive/MyDrive/train1"

testPath = r"/content/drive/MyDrive/test1"


In [ ]:
vg = VGG16(input_shape=imageSize + [3], weights='imagenet',include_top=False)

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
# don't train existing weights
for layer in vg.layers:
  layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vg.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=vg.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy'], run_eagerly=True
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(trainPath,
                                                 target_size = (224, 224),
                                                 batch_size =  30,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(testPath,
                                            target_size = (224, 224),
                                            batch_size = 30,
                                            class_mode = 'categorical')

Found 13175 images belonging to 5 classes.
Found 880 images belonging to 5 classes.


In [ ]:
training_set.class_indices

{'COVID19': 0, 'NORMAL': 1, 'NOTANXRAY': 2, 'PNEUMONIA': 3, 'TURBERCULOSIS': 4}

In [ ]:
len(training_set)

440

In [ ]:
len(test_set)

30

In [ ]:
import sys
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set)//40,
  validation_steps=len(test_set)//10)

<ipython-input-17-e708c8e11781>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
11/11 [==============================] - 906s 82s/step - loss: 1.0959 - accuracy: 0.7182 - val_loss: 1.4993 - val_accuracy: 0.7667
Epoch 2/10
11/11 [==============================] - 893s 81s/step - loss: 0.4014 - accuracy: 0.8909 - val_loss: 0.5468 - val_accuracy: 0.8000
Epoch 3/10
11/11 [==============================] - 875s 79s/step - loss: 0.1316 - accuracy: 0.9545 - val_loss: 0.4408 - val_accuracy: 0.8222
Epoch 4/10
11/11 [==============================] - 889s 81s/step - loss: 0.1975 - accuracy: 0.9212 - val_loss: 0.7534 - val_accuracy: 0.7444
Epoch 5/10
11/11 [==============================] - 909s 82s/step - loss: 0.1999 - accuracy: 0.9152 - val_loss: 0.4882 - val_accuracy: 0.8111
Epoch 6/10
11/11 [==============================] - 928s 84s/step - loss: 0.1576 - accuracy: 0.9455 - val_loss: 0.3792 - val_accuracy: 0.8444
Epoch 7/10
11/11 [==============================] - 931s 85s/step - loss: 0.1532 - accuracy: 0.9515 - val_loss: 0.2909 - val_accuracy: 0.9000
Epoch 

In [ ]:
model.save('xray.h5')